In [60]:
model_stemp_epic <- function (NL,
         ISWWAT,
         BD,
         DLAYR,
         DS,
         DUL,
         LL,
         NLAYR,
         TAMP,
         RAIN,
         TMA,
         CUMDPT,
         DSMID,
         SW,
         TAVG,
         TMAX,
         TMIN,
         TAV,
         SRFTEMP,
         NDays,
         TDL,
         WetDay,
         ST,
         DEPIR,
         BIOMAS,
         MULCHMASS,
         SNOW){

    TBD <- 0.0
    TLL <- 0.0
    TSW <- 0.0
    X2_PREV <- 0.0
    for( L in seq(1, NLAYR, 1)){
        TBD <- TBD + (BD[(L - 1)+1] * DLAYR[(L - 1)+1])
        TDL <- TDL + (DUL[(L - 1)+1] * DLAYR[(L - 1)+1])
        TLL <- TLL + (LL[(L - 1)+1] * DLAYR[(L - 1)+1])
        TSW <- TSW + (SW[(L - 1)+1] * DLAYR[(L - 1)+1])
    }
    ABD <- TBD / DS[(NLAYR - 1)+1]
    FX <- ABD / (ABD + (686.0 * exp(-(5.63 * ABD))))
    DP <- 1000.0 + (2500.0 * FX)
    WW <- 0.356 - (0.144 * ABD)
    B <- log(500.0 / DP)
    if (ISWWAT == 'Y')
    {
        PESW <- max(0.0, TSW - TLL)
    }
    else
    {
        PESW <- max(0.0, TDL - TLL)
    }
    if (NDays == 30)
    {
        for( I in seq(1, 29 + 1-1, 1)){
            WetDay[I - 1+1] <- WetDay[I + 1 - 1+1]
        }
    }
    else
    {
        NDays <- NDays + 1
    }
    if (RAIN + DEPIR > 1.E-6)
    {
        WetDay[NDays - 1+1] <- 1
    }
    else
    {
        WetDay[NDays - 1+1] <- 0
    }
    NWetDays <- sum(WetDay)
    WFT <- as.double(NWetDays) / as.double(NDays)
    CV <- (BIOMAS + MULCHMASS) / 1000.
    BCV1 <- CV / (CV + exp(5.3396 - (2.3951 * CV)))
    BCV2 <- SNOW / (SNOW + exp(2.303 - (0.2197 * SNOW)))
    BCV <- max(BCV1, BCV2)
    z <- SOILT_EPIC(NL, B, BCV, CUMDPT, DP, DSMID, NLAYR, PESW, TAV, TAVG, TMAX, TMIN, WetDay[NDays - 1+1], WFT, WW, TMA, X2_PREV, ST)
    TMA = z$TMA 
    X2_PREV = z$X2_PREV
    ST = z$ST
    SRFTEMP = z$SRFTEMP
    X2_AVG = z$X2_AVG
    return (list ("SRFTEMP" = SRFTEMP,"NDays" = NDays,"TDL" = TDL,"WetDay" = WetDay,"ST" = ST))
}



In [115]:
SOILT_EPIC <- function (NL,
         B,
         BCV,
         CUMDPT,
         DP,
         DSMID,
         NLAYR,
         PESW,
         TAV,
         TAVG,
         TMAX,
         TMIN,
         WetDay,
         WFT,
         WW,
         TMA,
         X2_PREV,
         ST){
    LAG <- 0.5
    WC <- max(0.01, PESW) / (WW * CUMDPT) * 10.0
    FX <- exp(B * ((1.0 - WC) / (1.0 + WC)) ^ 2)
    DD <- FX * DP
    if (WetDay > 0)
    {
        X2 <- WFT * (TAVG - TMIN) + TMIN
    }
    else
    {
        X2 <- WFT * (TMAX - TAVG) + TAVG + 2.
    }
    TMA[1 - 1+1] <- X2
    for( K in seq(5, 2, -1)){
        TMA[K] <- TMA[K - 1]
    }
    X2_AVG <- sum(TMA) / 5.0
    X3 <- (1. - BCV) * X2_AVG + (BCV * X2_PREV)
    SRFTEMP <- min(X2_AVG, X3)
    X1 <- TAV - X3
    for( L in seq(1, NLAYR, 1)){
        ZD <- DSMID[L] / DD
        F <- ZD / (ZD + exp(-.8669 - (2.0775 * ZD)))
        ST[L] <- LAG * ST[L] + ((1. - LAG) * (F * X1 + X3))
    }
    X2_PREV <- X2_AVG
    return (list ("TMA" = TMA,"X2_PREV" = X2_PREV,"ST" = ST,"SRFTEMP" = SRFTEMP,"X2_AVG" = X2_AVG))
}



In [118]:
init_stemp_epic <- function (NL,
         ISWWAT,
         BD,
         DLAYR,
         DS,
         DUL,
         LL,
         NLAYR,
         TAMP,
         RAIN,
         SW,
         TAVG,
         TMAX,
         TMIN,
         TAV,
         DEPIR,
         BIOMAS,
         MULCHMASS,
         SNOW){
    TMA <- array(c(0),dim=c(5,1,1))
    DSMID <- array(c(0),dim=c(NL,1,1))
    WetDay <- array(c(0),dim=c(NL,1,1))
    ST <- array(c(0),dim=c(NL,1,1))
    SWI <- array(c(0),dim=c(NL,1,1))
    SWI <- SW
    TBD <- 0.0
    TLL <- 0.0
    TSW <- 0.0
    TDL <- 0.0
    CUMDPT <- 0.0
    X2_PREV <- 0.0
    for( L in seq(1, NLAYR + 1-1, 1)){
        DSMID[L - 1+1] <- CUMDPT + (DLAYR[(L - 1)+1] * 5.0)
        CUMDPT <- CUMDPT + (DLAYR[(L - 1)+1] * 10.0)
        TBD <- TBD + (BD[(L - 1)+1] * DLAYR[(L - 1)+1])
        TLL <- TLL + (LL[(L - 1)+1] * DLAYR[(L - 1)+1])
        TSW <- TSW + (SWI[(L - 1)+1] * DLAYR[(L - 1)+1])
        TDL <- TDL + (DUL[(L - 1)+1] * DLAYR[(L - 1)+1])
    }
    if (ISWWAT == 'Y')
    {
        PESW <- max(0.0, TSW - TLL)
    }
    else
    {
        PESW <- max(0.0, TDL - TLL)
    }
    ABD <- TBD / DS[(NLAYR - 1)+1]
    print("tttttt")
    print(DS[(NLAYR - 1)+1])
    FX <- ABD / (ABD + (686.0 * exp(-(5.63 * ABD))))
    DP <- 1000.0 + (2500.0 * FX)
    WW <- 0.356 - (0.144 * ABD)
    B <- log(500.0 / DP)
    for( I in seq(1, 5 + 1-1, 1)){
        TMA[I - 1+1] <- as.integer(TAVG * 10000.) / 10000.
    }
    X2_AVG <- TMA[(1 - 1)+1] * 5.0
    for( L in seq(1, NLAYR, 1)){
        ST[L - 1+1] <- TAVG
    }
    WFT <- 0.1
    WetDay  <- array(c(0), dim=c(30,1,1))
    NDays <- 0
    CV <- MULCHMASS / 1000.
    BCV1 <- CV / (CV + exp(5.3396 - (2.3951 * CV)))
    BCV2 <- SNOW / (SNOW + exp(2.303 - (0.2197 * SNOW)))
    BCV <- max(BCV1, BCV2)
    print("iiiiii")
    print(WW)
    for( I in seq(1, 8, 1)){
        z <- SOILT_EPIC(NL, B, BCV, CUMDPT, DP, DSMID, NLAYR, PESW, TAV, TAVG, TMAX, TMIN, 0, WFT, WW, TMA, X2_PREV, ST)
        TMA = z$TMA 
        X2_PREV = z$X2_PREV
        ST = z$ST
        SRFTEMP = z$SRFTEMP
        X2_AVG = z$X2_AVG
    }
    return (list ("TMA" = TMA,"CUMDPT" = CUMDPT,"DSMID" = DSMID,"SRFTEMP" = SRFTEMP,"NDays" = NDays,"TDL" = TDL,"WetDay" = WetDay,"ST" = ST))
}

In [121]:
    DYNAMIC = 1
    ISWWAT = 'Y'
    BD    = array(c(1.6),dim=c(4,1,1))
    DLAYR = array(c(10.0),dim=c(4,1,1))
    DS = array(c(0.0),dim=c(4,1,1))
    DS[1] =10.0
    DS[2] =20.0
    DS[3] =30.0
    DS[4] =40.0
    DUL   = array(c(0.3),dim=c(20,1,1))
    LL    = array(c(0.2),dim=c(20,1,1))
    NLAYR = 4
    MSALB = 0.13
    SRAD    = 20.0
    SW      = array(c(0.2),dim=c(20,1,1))
    TAVG    = 25.0
    TMAX    = 30.0
    TMIN    = 20.0
    XLAT    = 28.0
    TAV     = 20.0
    TAMP    = 10.0
    RAIN = 0.0
    MULCHMASS = 0.0
    SNOW = 0.0
    DEPIR = 0.0
    BIOMAS = 0.0
    NL = 20

    z= init_stemp_epic(NL,ISWWAT,BD,DLAYR, DS,DUL,LL,NLAYR,TAMP,RAIN,SW,TAVG,TMAX,TMIN,TAV, DEPIR,BIOMAS,MULCHMASS,SNOW)


[1] "tttttt"
[1] 40
[1] "iiiiii"
[1] 0.1256


In [122]:
z

$TMA
, , 1

     [,1]
[1,] 27.5
[2,] 27.5
[3,] 27.5
[4,] 27.5
[5,] 27.5


$CUMDPT
[1] 400

$DSMID
, , 1

      [,1]
 [1,]   50
 [2,]  150
 [3,]  250
 [4,]  350
 [5,]    0
 [6,]    0
 [7,]    0
 [8,]    0
 [9,]    0
[10,]    0
[11,]    0
[12,]    0
[13,]    0
[14,]    0
[15,]    0
[16,]    0
[17,]    0
[18,]    0
[19,]    0
[20,]    0


$SRFTEMP
[1] 27.5

$NDays
[1] 0

$TDL
[1] 12

$WetDay
, , 1

      [,1]
 [1,]    0
 [2,]    0
 [3,]    0
 [4,]    0
 [5,]    0
 [6,]    0
 [7,]    0
 [8,]    0
 [9,]    0
[10,]    0
[11,]    0
[12,]    0
[13,]    0
[14,]    0
[15,]    0
[16,]    0
[17,]    0
[18,]    0
[19,]    0
[20,]    0
[21,]    0
[22,]    0
[23,]    0
[24,]    0
[25,]    0
[26,]    0
[27,]    0
[28,]    0
[29,]    0
[30,]    0


$ST
, , 1

          [,1]
 [1,] 25.80490
 [2,] 23.25960
 [3,] 21.76815
 [4,] 20.96581
 [5,]  0.00000
 [6,]  0.00000
 [7,]  0.00000
 [8,]  0.00000
 [9,]  0.00000
[10,]  0.00000
[11,]  0.00000
[12,]  0.00000
[13,]  0.00000
[14,]  0.00000
[15,]  0.00000
[16,]  0.00000
[17,]  0.00000
[18,]  0.00000
[19,]  0.00000
[20,]  0.00000

In [46]:
a=z[2][1]

In [47]:
a


$CUMDPT
[1] 400

In [81]:
 DSMID <- array(c(50,150,250,350),dim=c(4,1,1))

In [84]:
DSMID[1]+10/20

[1] 50.5

In [86]:
ZD <- DSMID[2] / 20

In [87]:
List[a,b]=

[1] 7.5

In [93]:
 for (j in seq(1,3,1)) print(DSMID[3])

[1] 250
[1] 250
[1] 250
